In [1]:
import pandas as pd
import numpy as np
import pathlib
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import RocCurveDisplay
from sklearn.decomposition import NMF
import warnings
warnings.filterwarnings("ignore")
from sklearn.mixture import GaussianMixture
import os
import ot
import pickle
import argparse
import Levenshtein
import itertools
from helper_functions import *

##### input args
PROJECT = "gs-mrd"
release_version = "06062024"
merge_version = "20240914"

##### configurations/paths
path_to_main_src = "/media/hieunguyen/HNSD01/src/gs-mrd"
path_to_merge_samples = f"{path_to_main_src}/all_samples/{merge_version}"
path_to_model_files = f"{path_to_main_src}/model_files/{release_version}"

##### metadata
motif_order = pd.read_csv("motif_order.csv").motif_order.to_list()
metadata = pd.read_csv(os.path.join(path_to_merge_samples, "EM_batch_metadata.csv"))
metadata["SampleID2"] = metadata["SampleID"].apply(lambda x: x.split("-")[1])
data_metadata = metadata.copy()

general_metadata = pd.read_excel("All Samples GW_MRD_010924.modified.xlsx", index_col = [0])
metadata = metadata[metadata["SampleID2"].isin(general_metadata["SampleID"].to_list())]

metadata["Cancer"] = metadata["SampleID"].apply(lambda x: general_metadata[general_metadata["SampleID"] == x.split("-")[1]].Cancer.unique()[0])
metadata["True label"] = metadata["SampleID"].apply(lambda x: general_metadata[general_metadata["SampleID"] == x.split("-")[1]]["True label"].unique()[0])

##### predict new samples only, remove "release_version" samples
metadata = metadata[metadata["Group_RUN"] != release_version]
featuredf = dict()
for input_feature in ["EM", "FLEN", "NUCLEOSOME", "IchorCNA"]:
    tmpdf = pd.read_csv(f"{path_to_merge_samples}/{input_feature}_features.csv")
    tmpdf = tmpdf[tmpdf["SampleID"].isin(metadata["SampleID"].to_list())]
    if input_feature == "EM":
        featuredf[input_feature] = tmpdf[["SampleID"] + motif_order].copy()
    else:
        featuredf[input_feature] = tmpdf.copy()


In [11]:
metadata[metadata["SampleID2"] == "UHAA48" ]

,SampleID,RUN,Group_RUN,SampleID2,Cancer,True label
781,4-UHAA48,R5498,cmc_MRD,UHAA48,Lung,-
1427,4-UHAA48,batch_240724,from_ECD_WGS,UHAA48,Lung,-
1447,6-UHAA48,batch_270824,from_ECD_WGS,UHAA48,Lung,-


In [9]:
metadata[metadata.SampleID2.duplicated()]

,SampleID,RUN,Group_RUN,SampleID2,Cancer,True label
521,3-ZMG138C,R5669,cmc_MRD,ZMG138C,Gastric,-
587,74-GWCT,R5540,cmc_MRD,GWCT,HCC,+
597,30-GWCT,R5633,cmc_MRD,GWCT,HCC,+
659,28-GWCT,R5662,cmc_MRD,GWCT,HCC,+
663,210-GWCT,R5706,cmc_MRD,GWCT,HCC,+
...,...,...,...,...,...,...
1446,3-ZMB009B,batch_270824,from_ECD_WGS,ZMB009B,Breast,-
1447,6-UHAA48,batch_270824,from_ECD_WGS,UHAA48,Lung,-
1448,5-ZMG138C,batch_270824,from_ECD_WGS,ZMG138C,Gastric,-
1449,4-ZMG096C,batch_270824,from_ECD_WGS,ZMG096C,Gastric,-
